In [22]:
# %pip install ollama
# %pip install --upgrade gradio

In [27]:
import gradio as gr
import random
import ollama

# Player 및 LLMPlayer 클래스 정의
class Player:
    def __init__(self, name: str):
        self.name = name
        self.character = ""

    def check_character(self, character):
        return self.character == character

class LLMPlayer(Player):
    MODEL_NAME = "EEVE-Korean-10.8B"
    SYSTEM_PROMPT = (
        "<|im_start|>system\n"
        "당신은 양세찬 게임 전문 참가자입니다. 반드시 다음 규칙을 따라야 합니다:\n"
        "1. 질문 생성 규칙\n"
        "- 예/아니오로 답변 가능한 형태\n"
        "- 캐릭터 이름 직접 언급 금지\n"
        "- 주제와 관련된 내용\n"
        "2. 답변 규칙\n"
        "- 반드시 사실에 기반해 답변\n"
        "- '예', '아니오'로만 답변\n"
        "- 추가 설명 없이 3단어 이내로 응답\n"
        "3. 추측 규칙\n"
        "- 반드시 한 명의 '인물'로만 답변\n"
        "- 불확실하면 '모름'이라고 답변\n"
        "<|im_end|>\n"
    )

    def __init__(self, name):
        super().__init__(name)
        self.messages = [{"role": "system", "content": self.__class__.SYSTEM_PROMPT}]

    def get_Q_res(self, topic, character):
        prompt = (
            "<|im_start|>user\n"
            f"주제: {topic}\n"
            f"내 캐릭터: {character}\n"
            "예/아니오로 답할 수 있는 질문을 하나만 생성하세요. 캐릭터 이름 직접 언급 금지.<|im_end|>\n"
            "<|im_start|>assistant\n"
        )
        self.messages.append({"role": "user", "content": prompt})
        q_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=self.messages,
            options={"stop": ["<|im_start|>", "<|im_end|>", "<|eot_id|>"], "temperature": 0.7}
        )['message']['content'].strip()
        self.messages.append({"role": "assistant", "content": q_res})
        return q_res

    def get_A_res(self, question, character):
        prompt = (
            "<|im_start|>user\n"
            f"질문: '{question}'\n"
            f"캐릭터: '{character}'\n"
            "1. '예'/'아니오' 또는 단답형으로만 답변\n"
            "2. 캐릭터 이름 언급 금지\n"
            "3. 사실에 기반한 대답<|im_end|>\n"
            "<|im_start|>assistant\n"
        )
        a_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=[{"role": "system", "content": self.__class__.SYSTEM_PROMPT}, {"role": "user", "content": prompt}],
            options={"stop": ["<|im_start|>", "<|im_end|>", "<|eot_id|>"], "temperature": 0.3}
        )['message']['content'].strip()
        return a_res

    def get_R_res(self, info):
        prompt = (
            "<|im_start|>user\n"
            "현재까지의 정보만으로 내 캐릭터가 누구인지 추측하세요.\n"
            "1. 반드시 한 명의 '인물' 또는 '가상인물'로 답변\n"
            "2. 모르는 경우 '모름'으로 답변\n"
            "3. 질문형식 답변 금지\n"
            "4. '?' 포함 금지<|im_end|>\n"
            "<|im_start|>assistant\n"
        )
        self.messages.append({"role": "user", "content": prompt + info})
        r_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=self.messages,
            options={"stop": ["<|im_start|>", "<|im_end|>", "<|eot_id|>"], "temperature": 0.3}
        )['message']['content'].strip()
        self.messages.append({"role": "assistant", "content": r_res})
        return r_res

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

TOPICS = ["연예인", "영화 인물", "드라마 인물", "위인", "캐릭터"]
CHARACTERS = {
    "연예인": [
        "유재석", "아이유", "BTS 진", "블랙핑크 제니", "송강호", "김태리", "이영자", "박보검", "전지현", "이병헌",
        "공유", "한효주", "김수현", "김고은", "박명수", "이승기", "박서준", "정해인", "차은우", "수지"
    ],
    "영화 인물": [
        "해리 포터", "헤르미온느 그레인저", "론 위즐리", "인디아나 존스", "제임스 본드", "다스 베이더", "루크 스카이워커", "한 솔로", "조커", "배트맨",
        "포레스트 검프", "토니 스타크", "캡틴 아메리카", "엘사", "올라프", "도로시", "토토", "한니발 렉터", "토니 몬타나", "존 윅"
    ],
    "드라마 인물": [
        "김신", "지은탁", "이헌", "고애신", "도민준", "천송이", "성덕선", "최택", "박새로이", "조이서",
        "김주원", "길라임", "이강", "채송화", "이준혁", "차수현", "윤세리", "리정혁", "장만월", "구찬성"
    ],
    "위인": [
        "세종대왕", "이순신", "간디", "아인슈타인", "링컨", "나폴레옹", "클레오파트라", "칭기즈칸", "넬슨 만델라", "마리 퀴리",
        "플라톤", "아리스토텔레스", "에디슨", "테슬라", "다윈", "갈릴레이", "코페르니쿠스", "뉴턴", "파스퇴르", "멘델"
    ],
    "캐릭터": [
        "도라에몽", "피카츄", "뽀로로", "미키마우스", "슈퍼마리오", "소닉", "톰", "제리", "스폰지밥", "심슨",
        "짱구", "곰돌이 푸", "미니언", "토토로", "포뇨", "둘리", "고길동", "베지터", "카카로트", "루피"
    ]
}

game_state = {
    "players": {},
    "topic": "",
    "current_player_idx": 0,
    "round": 1,
    "game_started": False,
    "logs": [],
    "current_player": "",
    "winner": None
}

def combine_logs(logs):
    return "\n".join(logs)

def create_players(player_count):
    players = {}
    players["user"] = Player("user")
    for i in range(1, player_count):
        name = f"AI-{i}"
        players[name] = LLMPlayer(name)
    return players

def random_characters(topic, player_count):
    return random.sample(CHARACTERS[topic], k=player_count)

def start_game(topic, player_count):
    game_state["topic"] = topic
    game_state["players"] = create_players(player_count)
    game_state["round"] = 1
    game_state["current_player_idx"] = 0
    game_state["current_player"] = "user"
    game_state["game_started"] = True
    game_state["logs"] = []
    game_state["winner"] = None

    characters = random_characters(topic, player_count)
    for i, (name, player) in enumerate(game_state["players"].items()):
        player.character = characters[i]

    logs = []
    logs.append(f"게임 시작!!! 주제: '{topic}'")
    logs.append("각 플레이어에게 캐릭터를 할당했습니다.")
    for name, player in game_state["players"].items():
        if name == "user":
            logs.append(f"- {name} : {player.character}")
        else:
            logs.append(f"- {name} : {player.character}")
    logs.append(f"\n--- {game_state['round']}라운드 ---")
    logs.append(f"\n[{game_state['current_player']}의 차례]")
    logs.append("질문을 입력하세요.")
    game_state["logs"] = logs

    combined_log = combine_logs(logs)
    return (
        "게임이 시작되었습니다. 당신의 차례입니다!",
        combined_log,
        "",  # 입력창 초기화
        gr.Button("입력하기")  # 입력창 활성화
    )

def user_input_handler(user_input):
    if not game_state["game_started"] or game_state["winner"]:
        return "게임이 시작되지 않았거나 이미 종료되었습니다.", combine_logs(game_state["logs"]), "", False

    logs = game_state["logs"].copy()
    current_player = game_state["current_player"]

    # 내 차례일 때
    if current_player == "user":
        # 질문 단계
        if not logs or logs[-1].endswith("질문을 입력하세요."):
            logs.append(f"user 질문 >> {user_input}")
            for name, player in game_state["players"].items():
                if name == "user":
                    continue
                answer = player.get_A_res(
                    user_input,
                    game_state["players"]["user"].character
                )
                logs.append(f"{name} 답변 >> {answer}")
            logs.append("정답을 입력하세요.")
            game_state["logs"] = logs
            return "질문이 등록되었습니다. 정답을 입력하세요!", combine_logs(logs), "", True

        # 정답 단계
        elif logs[-1].endswith("정답을 입력하세요."):
            logs.append(f"user 정답 >> {user_input}")
            if user_input and user_input.strip() == game_state["players"]["user"].character:
                logs.append(f"🎉 user님이 정답을 맞췄습니다! 게임 종료")
                game_state["winner"] = "user"
                game_state["logs"] = logs
                return "정답입니다! 게임 종료", combine_logs(logs), "", False
            else:
                logs.append("정답이 아닙니다. 다음 플레이어로 넘어갑니다.")
                player_names = list(game_state["players"].keys())
                game_state["current_player_idx"] = (game_state["current_player_idx"] + 1) % len(player_names)
                game_state["current_player"] = player_names[game_state["current_player_idx"]]
                if game_state["current_player_idx"] == 0:
                    game_state["round"] += 1
                    logs.append(f"\n--- {game_state['round']}라운드 ---")
                logs.append(f"\n[{game_state['current_player']}의 차례]")
                game_state["logs"] = logs
                return ai_turn(), combine_logs(game_state["logs"]), "", True

    # AI 차례일 때 (user가 입력하면 무시)
    return "AI 차례입니다. 잠시만 기다려주세요.", combine_logs(logs), "", False

def ai_turn():
    logs = game_state["logs"]
    current_player = game_state["current_player"]
    if current_player == "user" or game_state["winner"]:
        if game_state["winner"]:
            return "게임이 종료되었습니다."
        else:
            logs.append("질문을 입력하세요.")
            return "당신의 차례입니다. 질문을 입력하세요."
    ai_player = game_state["players"][current_player]
    ai_question = ai_player.get_Q_res(game_state["topic"], ai_player.character)
    logs.append(f"{current_player} 질문 >> {ai_question}")
    if "user" in game_state["players"]:
        logs.append("질문에 대한 답변을 입력하세요.")
        game_state["logs"] = logs
        return f"{current_player}가 질문했습니다. 답변을 입력하세요."
    return "AI 턴 처리 완료"

def user_input_handler_ai_answer(user_input):
    logs = game_state["logs"].copy()
    current_player = game_state["current_player"]
    if logs and logs[-1].endswith("질문에 대한 답변을 입력하세요."):
        logs.append(f"user 답변 >> {user_input}")
        player_names = list(game_state["players"].keys())
        game_state["current_player_idx"] = (game_state["current_player_idx"] + 1) % len(player_names)
        game_state["current_player"] = player_names[game_state["current_player_idx"]]
        if game_state["current_player_idx"] == 0:
            game_state["round"] += 1
            logs.append(f"\n--- {game_state['round']}라운드 ---")
        logs.append(f"\n[{game_state['current_player']}의 차례]")
        game_state["logs"] = logs
        return ai_turn(), combine_logs(game_state["logs"]), "", True
    return user_input_handler(user_input)

def unified_input_handler(user_input):
    logs = game_state["logs"]
    if logs and logs[-1].endswith("질문에 대한 답변을 입력하세요."):
        return user_input_handler_ai_answer(user_input)
    return user_input_handler(user_input)

with gr.Blocks(title="양세찬 게임") as demo:
    gr.Markdown("# 양세찬 게임 - 캐릭터 맞추기")

    with gr.Row():
        topic_dropdown = gr.Dropdown(choices=TOPICS, label="주제 선택", value=TOPICS[0])
        player_count_slider = gr.Slider(minimum=2, maximum=4, step=1, value=3, label="플레이어 수 (사용자 포함)")
        start_button = gr.Button("게임 시작")

    status_text = gr.Textbox(label="게임 상태", interactive=False)
    
    with gr.Tab("게임 로그"):
        log_textbox = gr.Textbox(label="전체 로그", interactive=False, lines=30)
        unified_input = gr.Textbox(label="입력", placeholder="질문, 답변 또는 정답을 입력하세요", interactive=True)
        submit_button = gr.Button("입력하기")

    start_button.click(
        fn=start_game,
        inputs=[topic_dropdown, player_count_slider],
        outputs=[status_text, log_textbox, unified_input, submit_button]
    )

    submit_button.click(
        fn=unified_input_handler,
        inputs=[unified_input],
        outputs=[status_text, log_textbox, unified_input, submit_button]
    )

if __name__ == "__main__":
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7879
* Running on public URL: https://aea909d465518bb847.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
import ollama



class Player:
    def __init__(self, name: str):
        self.name = name
        self.character = ""

    def check_character(self, character):
        return (self.character == character)



class LLMPlayer(Player):
    MODEL_NAME="EEVE-Korean-10.8B"
    SYSTEM_PROMPT = ("""
        당신은 양세찬 게임 전문 참가자입니다. 반드시 다음 규칙을 따라야 합니다:

        1. 질문 생성 규칙
        - 예/아니오로 답변 가능한 형태
        - 캐릭터 이름 직접 언급 금지
        - 주제와 관련된 내용

        2. 답변 규칙
        - 반드시 사실에 기반해 답변
        - '예', '아니오'로만 답변
        - 추가 설명 없이 3단어 이내로 응답

        3. 추측 규칙
        - 반드시 한 명의 '인물'로만 답변
        - 불확실하면 '모름'이라고 답변
    """)
    


    def __init__(self, name):
        super().__init__(name)        
        self.messages = [{"role": "system", "content": self.__class__.SYSTEM_PROMPT}]



    # 자신 캐릭터 관련 질문 (Question)
    def get_Q_res(self, content):
        self.messages.append({"role": "user", "content": content})
       
        q_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=self.messages,
        )['message']['content']

        self.messages.append({"role": "assistant", "content": q_res})

        return q_res
    


    # 상대 질문에 대한 답 (Answer)
    def get_A_res(self, content):
        a_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=[
                {"role": "system", "content": self.__class__.SYSTEM_PROMPT},
                {"role": "user", "content": content}
            ],
        )['message']['content']

        return a_res
    


    # 내 캐릭터 유추 (Result)
    def get_R_res(self, content):
        self.messages.append({"role": "user", "content": content})
       
        r_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=self.messages,
        )['message']['content']

        self.messages.append({"role": "assistant", "content": r_res})

        return r_res



    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

In [4]:
# 주제 목록
TOPICS = ["연예인", "영화 인물", "드라마 인물", "위인", "캐릭터"]

# 각 주제별 예시 인물
CHARACTERS = {
    "연예인": [
        "유재석", "아이유", "BTS 진", "블랙핑크 제니", "송강호", "김태리", "이영자", "박보검", "전지현", "이병헌",
        "공유", "한효주", "김수현", "김고은", "박명수", "이승기", "박서준", "정해인", "차은우", "수지"
    ],
    "영화 인물": [
        "해리 포터", "헤르미온느 그레인저", "론 위즐리", "인디아나 존스", "제임스 본드", "다스 베이더", "루크 스카이워커", "한 솔로", "조커", "배트맨",
        "포레스트 검프", "토니 스타크", "캡틴 아메리카", "엘사", "올라프", "도로시", "토토", "한니발 렉터", "토니 몬타나", "존 윅"                
    ],
    "드라마 인물": [
        "김신", "지은탁", "이헌", "고애신", "도민준", "천송이", "성덕선", "최택", "박새로이", "조이서",
        "김주원", "길라임", "이강", "채송화", "이준혁", "차수현", "윤세리", "리정혁", "장만월", "구찬성"
    ],
    "위인": [
        "세종대왕", "이순신", "간디", "아인슈타인", "링컨", "나폴레옹", "클레오파트라", "칭기즈칸", "넬슨 만델라", "마리 퀴리",
        "플라톤", "아리스토텔레스", "에디슨", "테슬라", "다윈", "갈릴레이", "코페르니쿠스", "뉴턴", "파스퇴르", "멘델"
    ],
    "캐릭터": [
        "도라에몽", "피카츄", "뽀로로", "미키마우스", "슈퍼마리오", "소닉", "톰", "제리", "스폰지밥", "심슨",
        "짱구", "곰돌이 푸", "미니언", "토토로", "포뇨", "둘리", "고길동", "베지터", "카카로트", "루피"
    ]
}

In [5]:
import random

def random_topic():    
    return random.choice(TOPICS)


def random_characters(topic, player_count):
    print(CHARACTERS[topic])
    return random.choices(CHARACTERS[topic], k=player_count)

In [6]:
def create_players(player_count):
    players = {}

    # User
    players["user"] = Player("user")

    # AI
    for i in range(1, player_count):
        name = f"ai-{i}"
        players[name] = LLMPlayer(name)
    
    return players



# 게임 준비
player_count = 3
players = create_players(player_count)

In [7]:
# 주제 및 캐릭터 할당
topic = random_topic()
characters = random_characters(topic, player_count)

# 캐릭터 할당
for (player, character) in zip(players.values(), characters):
    player.character = character

['해리 포터', '헤르미온느 그레인저', '론 위즐리', '인디아나 존스', '제임스 본드', '다스 베이더', '루크 스카이워커', '한 솔로', '조커', '배트맨', '포레스트 검프', '토니 스타크', '캡틴 아메리카', '엘사', '올라프', '도로시', '토토', '한니발 렉터', '토니 몬타나', '존 윅']


In [8]:
# PROMPT 정의
QUESTION_PROMPT = (
    "다음 조건에 맞는 질문을 생성하세요:\n"
    "1. 내 캐릭터를 추론할 수 있는 내용\n"
    "2. 캐릭터 이름직접 언급 금지\n"
    "3. '예/아니오' 혹은 단답형으로 답변 가능할 수 있는 내용\n\n"

    "예시:\n"
    "- 예.\n"
    "- 네.\n"
    "- 아니오.\n"
    "- 아뇨.\n"
    "- 남자입니까?\n"
    "- 여자입니까?\n"
    "- 한국인입니까?\n"
    "- 외국인입니까?\n"
)

ANSWER_PROMPT = (
    "질문 '{question}'에 대해 캐릭터('{character}')가 맞는지 답변:\n"
    "1. '예'/'아니오' 또는 '단답형'으로만 답변\n"
    "2. 캐릭터 이름('{character}')을 응답에 포함시키지 말 것\n"
    "3. 사실에 기반한 대답\n\n"

    "예시:\n"
    "- 예, 밀짚모자를 씁니다.\n"
    "- 아니오, 미국드라마입니다.\n"
    "- 모름\n"
    "- 네, 맞아요.\n"
    "- 아니요, 아닙니다.\n"
    "- 남자예요.\n"
    "- 여자예요.\n"
    "- 한국인이에요.\n"
    "- 외국인이에요.\n"
)

GUESS_PROMPT = (
    "현재까지의 정보로 '내 캐릭터가 누구인지 추측'하세요:\n"
    "1. 반드시 한 명의 '인물' 또는 '가상인물'로 답변\n"
    "2. 모를 경우 '모름'\n\n"

    "예시:\n"
    "- 유재석\n"
    "- 해리포터\n"
    "- 둘리\n"
    "- 김구\n"
    "- 박효신\n"
    "- 모름\n"
    "- 포레스트 검프\n"
    "- 토니 스타크\n"
    "- 캡틴 아메리카\n"
    "- 길라임\n"
    "- 이강\n"
    "- 채송화\n"
    "- 플라톤\n"
    "- 아리스토텔레스\n"
    "- 에디슨\n"
    "- 테슬라\n"
    "- 짱구\n"
    "- 곰돌이 푸\n"
    "- 미니언\n"
    "- 토토로\n"
)


def play_game(players, topic, max_rounds=20):
    print(f"게임 시작!!!\n>>>> 주제 : '{topic}'\n")
    print("각 플레이어에게 캐릭터를 할당했했습니다.")
    for (name, player) in players.items():
        print(f"- {name} : {player.character}")
    
    winner = None
    round_num = 1

    while round_num <= max_rounds and winner is None:
        print(f"\n--- {round_num}라운드 ---")
        
        for (player_name, player) in players.items():
            player = players[player_name]
            print(f"\n[{player_name}의 차례]")
            
            # 질문
            question=""
            if (player_name == "user"):
                question = input("내 캐릭터에 대한 질문 >> ")
            else:
                question = player.get_Q_res(QUESTION_PROMPT)
            print(f"'{player_name}' 질문 >> {question}")
            
            # 답변
            for (other_name, other_player) in players.items():
                # 질문한 당사자는 건너뜀
                if (other_name == player_name):
                    continue
                
                answer=""
                if (other_name == "user"):
                    answer = input(f"{player_name}의 질문에 답변 >> ")
                else:
                    answer = other_player.get_A_res(
                        ANSWER_PROMPT.format(
                            character=player.character,
                            question=question
                        ),
                    )
                print(f"{other_name} 답변 >> {answer}")

            
            # 정답
            guess=""
            if (player_name == "user"):
                guess = input("정답 (패스 Enter) : ")
            else:
                guess = player.get_R_res(GUESS_PROMPT)
            print(f"{player_name} 정답 >> {guess}")


            # 정답 판별
            if guess and guess.strip() == player.character:
                print(f"🎉 {player_name}님이 정답을 맞췄습니다! 게임 종료")
                winner = player_name
                break
            else:
                print(f"{player_name}님은 정답을 맞추지 못했습니다.")

        round_num += 1

    if winner:
        print(f"\n게임 승자: {winner} (정답: {players[winner].character})")
    else:
        print("\n아쉽게도 아무도 정답을 맞히지 못했습니다.")


# 게임 플레이 시작
play_game(players, topic)

게임 시작!!!
>>>> 주제 : '영화 인물'

각 플레이어에게 캐릭터를 할당했했습니다.
- user : 다스 베이더
- ai-1 : 조커
- ai-2 : 포레스트 검프

--- 1라운드 ---

[user의 차례]


내 캐릭터에 대한 질문 >>  ㅎㅇ


'user' 질문 >> ㅎㅇ


ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

In [ ]:
# 